In [1]:
import copy
import itertools as its
import math
import os
import pathlib
from typing import Dict, List, Optional
from collections import Counter, defaultdict

import networkx as nx
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from aoc import sim_new as sim, testing, util

twopi = 2 * math.pi

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec10.txt'

In [2]:
def get_asteroids(text):
    return [
        (x, y)
        for y, line in enumerate(text.strip().split('\n'))
        for x, c in enumerate(line.strip())
        if c == '#'
    ]

In [3]:
test_input1 = """\
......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...#..#.
.#....####"""
test_asteroids1 = get_asteroids(test_input1)

test_input2 = """\
.#..#
.....
#####
....#
...##"""
test_asteroids2 = get_asteroids(test_input2)

In [4]:
asteroids = get_asteroids(INPUT_PATH.read_text())

In [5]:
def get_answer(asteroids):
    vis_by_base = {}
    for base in asteroids:
        vis_by_slope = defaultdict(list)
        vis_by_base[base] = vis_by_slope
        for other in asteroids:
            if other == base:
                continue

            # Get the slope; Note that sign matters
            slope_x_lg = base[0] - other[0]
            slope_y_lg = base[1] - other[1]
            if slope_y_lg == 0:
                slope_x = 1 if slope_x_lg > 0 else -1
                slope_y = 0
            elif slope_x_lg == 0:
                slope_x = 0
                slope_y = 1 if slope_y_lg > 0 else -1
            else:
                d = util.gcd(slope_x_lg, slope_y_lg)
                slope_x = slope_x_lg // d
                slope_y = slope_y_lg // d
            slope = (slope_x, slope_y)
            vis_by_slope[slope].append((other[0], other[1]))
    return vis_by_base

In [6]:
assert max(len(v) for v in get_answer(test_asteroids1).values()) == 33
assert max(len(v) for v in get_answer(test_asteroids2).values()) == 8

In [7]:
print(f'The answer to part 1 is {max(len(v) for v in get_answer(asteroids).values())}')

The answer to part 1 is 286


In [8]:
ans = get_answer(asteroids)
station = [key for key, val in ans.items() if len(val) == 286][0]

In [9]:
viz = ans[station]
for val in viz.values():
    val.sort(key=lambda other: -util.dist(other, station))

# Sort slopes by angle; note the -x here is because left/right hand rule issues
slope_viz = sorted((math.atan2(-x, y) % twopi, (x, y)) for x, y in viz)
counter = 0

for theta, slope in its.cycle(slope_viz):
    if viz[slope]:
        point = viz[slope].pop()
        counter += 1
        if counter == 200:
            print(f'The answer to part 2 is {100 * point[0] + point[1]}')
            break

The answer to part 2 is 504
